In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import Parallel, delayed

In [3]:
grids = "/data/sjammi6/thesisproject/updates/BlackHoleDiskProject/InitialData/routine1/grids_bh_disk_patrik"

# Grid Data

each line corresponds to a different grid with the following information:
- x_min
- y_min
- z_min
- dx
- dy
- dz
- Nx
- Ny
- Nz
- MPI_ID

In [4]:
lines = []
with open(grids, "r") as f:
    lines = f.readlines()

In [5]:
lines[0].split()

['./read_bin_bhdisk',
 '1.9840100000000001e+02',
 '6.4000000000000043e+01',
 '3.9680000000000001e+02',
 '6.4000000000000004e+00',
 '6.4000000000000004e+00',
 '6.4000000000000004e+00',
 '41',
 '37',
 '39',
 '-979427380']

In [6]:
grid_data = [
    l.split()[1:]
    for l in lines
]
grid_data = np.array(grid_data)
grid_data.reshape((1280, 10))

array([['1.9840100000000001e+02', '6.4000000000000043e+01',
        '3.9680000000000001e+02', ..., '37', '39', '-979427380'],
       ['1.1520100000000002e+02', '4.1600000000000037e+01',
        '2.2400000000000000e+02', ..., '40', '47', '1510771035'],
       ['5.7601000000000013e+01', '2.0800000000000036e+01',
        '1.1200000000000001e+02', ..., '40', '47', '-877886181'],
       ...,
       ['-5.7989999999999888e+00', '-5.7999999999999652e+00',
        '-1.9999999999999929e-01', ..., '49', '33', '44339239'],
       ['-2.8989999999999885e+00', '-2.8999999999999648e+00',
        '-1.0000000000000142e-01', ..., '49', '33', '38009956'],
       ['-1.4489999999999883e+00', '-1.4499999999999647e+00',
        '-5.0000000000000711e-02', ..., '49', '33', '35345179']],
      dtype='<U23')

In [7]:
grid_data[:, 6:9][0]

array(['41', '37', '39'], dtype='<U23')

In [8]:
grid_datav2 = np.column_stack((grid_data[:, :6].astype(np.float64), grid_data[:, 6:9].astype(np.int64), grid_data[:, 9].astype(np.str_)))
print(grid_datav2.shape)
print(grid_datav2[0])

(1280, 10)
['198.401' '64.00000000000004' '396.8' '6.4' '6.4' '6.4' '41' '37' '39'
 '-979427380']


NameError: name 'gridcols' is not defined

In [11]:
gridcols = ["x_min", "y_min", "z_min", "dx", "dy", "dz", "Nx", "Ny", "Nz", "MPI_ID"]

# Create the DataFrame with specified dtypes
data_dfgrid = {}
for i, d in enumerate(gridcols[:9]):
    data_dfgrid[d] = grid_datav2[:, i]

In [12]:
df_grids = pd.DataFrame(data_dfgrid, dtype=np.float64)
df_grids

,x_min,y_min,z_min,dx,dy,dz,Nx,Ny,Nz
0,198.401,64.00,396.80,6.4000,6.4000,6.4000,41.0,37.0,39.0
1,115.201,41.60,224.00,3.2000,3.2000,3.2000,45.0,40.0,47.0
2,57.601,20.80,112.00,1.6000,1.6000,1.6000,45.0,40.0,47.0
3,28.801,10.40,56.00,0.8000,0.8000,0.8000,45.0,40.0,47.0
4,14.401,5.20,28.00,0.4000,0.4000,0.4000,45.0,40.0,47.0
...,...,...,...,...,...,...,...,...,...
1275,-23.199,-23.20,-0.80,0.2000,0.2000,0.2000,47.0,49.0,33.0
1276,-11.599,-11.60,-0.40,0.1000,0.1000,0.1000,47.0,49.0,33.0
1277,-5.799,-5.80,-0.20,0.0500,0.0500,0.0500,47.0,49.0,33.0
1278,-2.899,-2.90,-0.10,0.0250,0.0250,0.0250,47.0,49.0,33.0


In [13]:
outputfilenames = ["CTS_bin-proc{}.d".format(str(i)) for i in grid_datav2[:, 9]]
df_grids["outputfilenames"] = outputfilenames
df_grids["MPI_ID"] = grid_datav2[:, 9]

In [14]:
df_grids["x_max"] = df_grids.x_min + df_grids.dx * df_grids.Nx
df_grids["y_max"] = df_grids.y_min + df_grids.dy * df_grids.Ny
df_grids["z_max"] = df_grids.z_min + df_grids.dz * df_grids.Nz
df_grids

,x_min,y_min,z_min,dx,dy,dz,Nx,Ny,Nz,outputfilenames,MPI_ID,x_max,y_max,z_max
0,198.401,64.00,396.80,6.4000,6.4000,6.4000,41.0,37.0,39.0,CTS_bin-proc-979427380.d,-979427380,460.8010,300.8000,646.4000
1,115.201,41.60,224.00,3.2000,3.2000,3.2000,45.0,40.0,47.0,CTS_bin-proc1510771035.d,1510771035,259.2010,169.6000,374.4000
2,57.601,20.80,112.00,1.6000,1.6000,1.6000,45.0,40.0,47.0,CTS_bin-proc-877886181.d,-877886181,129.6010,84.8000,187.2000
3,28.801,10.40,56.00,0.8000,0.8000,0.8000,45.0,40.0,47.0,CTS_bin-proc1232951467.d,1232951467,64.8010,42.4000,93.6000
4,14.401,5.20,28.00,0.4000,0.4000,0.4000,45.0,40.0,47.0,CTS_bin-proc380747062.d,380747062,32.4010,21.2000,46.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,-23.199,-23.20,-0.80,0.2000,0.2000,0.2000,47.0,49.0,33.0,CTS_bin-proc110307256.d,110307256,-13.7990,-13.4000,5.8000
1276,-11.599,-11.60,-0.40,0.1000,0.1000,0.1000,47.0,49.0,33.0,CTS_bin-proc60996709.d,60996709,-6.8990,-6.7000,2.9000
1277,-5.799,-5.80,-0.20,0.0500,0.0500,0.0500,47.0,49.0,33.0,CTS_bin-proc44339239.d,44339239,-3.4490,-3.3500,1.4500
1278,-2.899,-2.90,-0.10,0.0250,0.0250,0.0250,47.0,49.0,33.0,CTS_bin-proc38009956.d,38009956,-1.7240,-1.6750,0.7250


In [15]:
grid_metadata = dict(df_grids.describe())
df_grids.describe()

,x_min,y_min,z_min,dx,dy,dz,Nx,Ny,Nz,x_max,y_max,z_max
count,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000
mean,-26.351891,-26.463301,41.587168,1.278750,1.278750,1.278750,44.988281,43.960938,40.920313,28.436205,27.951309,90.705234
std,142.456958,146.013312,90.615580,1.956813,1.956813,1.956813,1.999770,5.712423,5.332558,141.541438,138.351560,148.710007
min,-639.999000,-640.000000,-25.600000,0.012500,0.012500,0.012500,37.000000,31.000000,30.000000,-390.399000,-377.600000,0.362500
25%,-23.199000,-24.000000,-0.100000,0.050000,0.050000,0.050000,45.000000,39.000000,37.000000,-2.999000,-3.700000,2.925000
50%,-1.449000,-1.450000,3.100000,0.300000,0.300000,0.300000,45.000000,40.000000,43.000000,2.026000,1.462500,20.400000
75%,1.826000,5.200000,33.600000,1.600000,1.600000,1.600000,47.000000,51.000000,47.000000,23.401000,33.600000,93.600000
max,409.601000,448.000000,454.400000,6.400000,6.400000,6.400000,47.000000,51.000000,47.000000,646.401000,646.400000,646.400000


In [16]:

x_min = grid_metadata["x_min"]["min"]
y_min = grid_metadata["y_min"]["min"]
z_min = grid_metadata["z_min"]["min"]
x_max = grid_metadata["x_max"]["max"]
y_max = grid_metadata["y_max"]["max"]
z_max = grid_metadata["z_max"]["max"]
dx = grid_metadata["dx"]["min"]
dy = grid_metadata["dy"]["min"]
dz = grid_metadata["dz"]["min"]

xarr_interpolated = np.arange(x_min, x_max, dx, dtype=np.float64)
yarr_interpolated = np.arange(y_min, y_max, dy, dtype=np.float64)
zarr_interpolated = np.arange(z_min, z_max, dz, dtype=np.float64)

In [17]:
print(dx)

0.0125


In [105]:

def gridmaker(x, y, z):
    gridcoordinates = np.array([x, y, z])
    return gridcoordinates

In [106]:
import numpy as np
from multiprocessing import Pool
from itertools import product

def create_grid_point(args):
    return list(args)

def parallel_grid_creation(xarr, yarr, zarr):
    with Pool() as pool:
        grid = pool.map(create_grid_point, product(xarr, yarr, zarr))
    return np.array(grid)


grid = parallel_grid_creation(xarr_interpolated, yarr_interpolated, zarr_interpolated)
print(grid)

: 